In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/test/rmgpy/test_data'}

In [3]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = ['Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 
                               'DFT_QCI_thermo',
                           'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
'SulfurLibrary', 'SulfurGlarborgH2S','SABIC_aromatics'],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'H_Abstraction',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [4]:
family = database.kinetics.families["H_Abstraction"]


In [10]:
family.clean_tree()

In [11]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=300)

# family.generate_tree(thermo_database=database.thermo,
#                      nprocs=1,
#                      new_fraction_threshold_to_reopt_node=0.25,
#                      max_batch_size=800,
#                      extension_iter_max=2,
#                      extension_iter_item_cap=100)

end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 1666.6666666666665 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([7.79,8.14,8.35,8.51,8.75,8.94,9.31],'cal/(mol*K)'), H298=(30.74,'kcal/mol'), S298=(54.54,'cal/(mol*

19770.297350406647


In [13]:
len(family.groups.entries)

18439

In [14]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

183.74981570243835


In [15]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 1666.6666666666665 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([7.79,8.14,8.35,8.51,8.75,8.94,9.31],'cal/(mol*K)'), H298=(30.74,'kcal/mol'), S298=(54.54,'cal/(mol*

175.09767627716064


In [16]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([4.9675,4.9675,4.9675,4.9675,4.9675,4.9674,4.9718],'cal/(mol*K)'), H298=(112.96,'kcal/mol'), S298=(36.6336,'cal/(mol*K)'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""N""", comment="""Thermo library: primaryNS""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 1666.6666666666665 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([7.79,8.14,8.35,8.51,8.75,8.94,9.31],'cal/(mol*K)'), H298=(30.74,'kcal/mol'), S298=(54.54,'cal/(mol*

160.07746076583862


In [17]:
len(templateRxnMap)

18439

In [18]:
family.clean_tree_rules()

In [ ]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3430: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/work/westgroup

In [ ]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

In [ ]:
errors.values()

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(np.abs(list(errors.values())),bins=30, density=True)
plt.title('Decision Tree Estimator',fontsize=18)
#plt.ylim(0,.4)
plt.xlim(0,15)
plt.ylabel('Probability density',fontsize=18)
plt.xlabel(r'$|Ln(k_{est}/k_{rxn})|$',fontsize=18)

In [ ]:
#family.save('/Users/mattjohnson/RMGCODE/RMG-database/input/kinetics/families/')

In [ ]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

In [ ]:
family.save(save_path)